In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers scipy numpy pandas datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from datasets import load_dataset

In [ ]:
# Loading...  game revieuws from google play store
df = pd.read_csv("reviews_processed.csv", delimiter=";")

print(f"Your dataset contains : {len(df['title'].unique())} games")
if df.duplicated().sum() == 0:
	print("no duplicates")
else:
	print(f"we deleting {df.duplicated().sum()} duplicates")
	df.drop_duplicates(inplace=True)

print(df.shape)

Your dataset contains : 374 games
no duplicates
(25384, 18)


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

inputs = tokenizer("Ok, I'm hopelessly hooked. Installed just to get coins in RR3, now I'm 180+ consecutive days playing, never had time to RR3 again. You can progress without paying real money (I did spend a few bucks, mostly from Google Rewards credits, and did it happily to pay back for a lot of fun), but of course then you'll need more patience.", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

'NEGATIVE'

Let's go FineTune distilBert to hope get the best accuracy !!

In [ ]:
dataset = load_dataset("csv", data_files="reviews_processed.csv", delimiter=";")
train_test = dataset["train"].train_test_split(test_size=0.2)
test_valid = train_test["test"].train_test_split(test_size=0.5)

train_dataset = train_test["train"]
valid_dataset = test_valid["train"]
test_dataset = test_valid["test"]

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

def preprocess_function(examples):
    # 0 as negative 1 as positive
    examples["label"] = [1 if x == "positive" else 0 for x in examples["sentiment"]]
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/20307 [00:00<?, ? examples/s]

Map:   0%|          | 0/2538 [00:00<?, ? examples/s]

Map:   0%|          | 0/2539 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.remove_columns(["text", "sentiment"])
train_dataset.set_format("torch")

valid_dataset = valid_dataset.remove_columns(["text", "sentiment"])
valid_dataset.set_format("torch")

test_dataset = test_dataset.remove_columns(["text", "sentiment"])
test_dataset.set_format("torch")


In [ ]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


In [ ]:
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

# f1 and accuracy
f1_metric = evaluate.load("f1")
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="binary")["f1"]
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": accuracy, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate = 1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    logging_steps=10,
    load_best_model_at_end=True,
    warmup_steps=600,
    metric_for_best_model="f1"  # f1 as best accuracy
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-30-c8ce4e50b719>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.062400,0.548461,0.870764,0.899325
2,0.031400,0.767244,0.866430,0.899137
3,0.047300,0.780386,0.870764,0.900425


TrainOutput(global_step=1905, training_loss=0.03705526620706939, metrics={'train_runtime': 662.67, 'train_samples_per_second': 91.933, 'train_steps_per_second': 2.875, 'total_flos': 2017511598380544.0, 'train_loss': 0.03705526620706939, 'epoch': 3.0})

In [ ]:
# After training is complete
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)
labels = predictions.label_ids
print(len(labels))

from sklearn.metrics import classification_report

# If you know your classes, for example 0 = negative, 1 = positive
target_names = ["negative", "positive"]

print(classification_report(labels, preds, target_names=target_names))


2539
              precision    recall  f1-score   support

    negative       0.81      0.85      0.83       893
    positive       0.92      0.89      0.90      1646

    accuracy                           0.88      2539
   macro avg       0.86      0.87      0.87      2539
weighted avg       0.88      0.88      0.88      2539

